In [ ]:
import ee
import geemap as gee
import geemap.eefolium as geemap
import json
import os
ee.Initialize()

In [ ]:
attributes = {'min': 0.0, 'max': 9000.0,
  'palette': [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301'
  ]}
sl_geom_boundary = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017").filter(ee.Filter.eq('country_na', 'Sri Lanka')).geometry()
x,y = sl_geom_boundary.centroid().getInfo()['coordinates'][1],sl_geom_boundary.centroid().getInfo()['coordinates'][0]
ndvi_dataset = ee.ImageCollection("MODIS/061/MOD13A1")
datasets = {}

for year in range(2015,2022):
    datasets[year] = ndvi_dataset.filterBounds(sl_geom_boundary).filter(ee.Filter.date(str(year) + '-01-01', str(year) + '-12-31'))

In [ ]:
Map = geemap.Map(center=(x,y), zoom=8)

for year, data in datasets.items():
    print(f"[+] processing... {year}")
    dataset = data.select('NDVI').map(lambda img: img.clip(sl_geom_boundary)).mean()
    Map.addLayer(dataset, attributes, 'NDVI ' + str(year))
    name = os.path.join("D:\\", str(year) + '.tif')
    # geemap.ee_export_image(dataset, filename="D:\\" + str(year) + ".tif", scale=550, region=sl_geom_boundary, file_per_band=True)
    geemap.ee_export_image_to_drive(dataset, fileNamePrefix="sl_ndvi_" + str(year), region=sl_geom_boundary, folder='export', scale=550)
Map